In [17]:
import pandas as pd
import pyodbc
from sqlalchemy import create_engine

In [18]:
conn_string = "DRIVER={ODBC Driver 17 for SQL Server};Server=localhost,1435;Uid=sa;Pwd=Wikipediapassword_01"
conn = pyodbc.connect(conn_string)

In [19]:
cursor = conn.cursor()

In [20]:
conn.autocommit = True

In [21]:
cursor.execute("SELECT name FROM sys.databases;")

In [22]:
cursor.fetchall()

[('master',), ('tempdb',), ('model',), ('msdb',)]

In [ ]:
cursor.execute("CREATE DATABASE Wikipediadb;")

In [31]:
cursor.execute("USE Wikipediadb;")

In [30]:
create_query_file = open('./sql/sqlserver_create_table.sql')
create_query = create_query_file.read()
create_query

'CREATE TABLE university (\n    id BIGINT IDENTITY(1,1) PRIMARY KEY,\n    country NVARCHAR(255) NOT NULL,\n    name NVARCHAR(255) NOT NULL,\n    founded INT NOT NULL,\n    type NVARCHAR(255) NOT NULL,\n    enrollment BIGINT NOT NULL,\n    link NVARCHAR(255) NOT NULL,\n    CONSTRAINT unique_combination UNIQUE (country, name)\n);'

In [32]:
cursor.execute(create_query)

In [33]:
engine = create_engine("mssql+pyodbc://", creator=lambda: conn)

In [42]:
pd.read_sql("SELECT * FROM university;", con=engine)

,id,country,name,founded,type,enrollment,link


In [35]:
df = pd.read_csv('universities-clean.csv')

In [44]:
# pd.read_sql("SELECT * FROM #staging WHERE country = 'France';", con=engine)

In [ ]:
df.to_sql("#staging", index=False, con=engine)

In [39]:
pd.read_sql("SELECT * FROM #staging WHERE country = 'France';", con=engine)

,Country,University,Founded,Type,Enrollment,Link
0,France,Centre national d'enseignement à distance,1939,Public,350000,https://en.wikipedia.org/wiki/Centre_national_...


In [40]:
pd.read_sql("SELECT * FROM university;", con=engine)

,id,country,name,founded,type,enrollment,link


In [46]:
merge_query_file = open('./sql/sqlserver_upsert.sql')
merge_query = merge_query_file.read()
merge_query

'MERGE INTO university target\nUSING #staging source\nON (source.University = target.name AND source.Country = target.country)\nWHEN MATCHED THEN UPDATE\nSET\n    target.founded=source.Founded,\n    target.type=source.Type,\n    target.enrollment=source.Enrollment,\n    target.link=source.Link\nWHEN NOT MATCHED THEN\nINSERT (country, name, founded, type, enrollment, link)\nVALUES (source.Country, source.University, source.Founded, source.Type, source.Enrollment, source.Link);'

In [47]:
cursor.execute(merge_query)

In [48]:
pd.read_sql("SELECT * FROM university;", con=engine)

,id,country,name,founded,type,enrollment,link
0,1,Albania,University of Tirana,1957,Public,35000,https://en.wikipedia.org/wiki/University_of_Ti...
1,2,Algeria,Constantine University,1978,Public,85000,https://en.wikipedia.org/wiki/List_of_universi...
2,3,Angola,Agostinho Neto University,1962,Public,29827,https://en.wikipedia.org/wiki/Agostinho_Neto_U...
3,4,Argentina,University of Buenos Aires,1821,Public,311175,https://en.wikipedia.org/wiki/University_of_Bu...
4,5,Australia,Monash University,1958,Public,73807,https://en.wikipedia.org/wiki/Monash_University
...,...,...,...,...,...,...,...
66,67,Turkey,Anadolu University,1958,Public,1969733,https://en.wikipedia.org/wiki/Anadolu_University
67,68,United Kingdom,Open University,1969,Public,253075,https://en.wikipedia.org/wiki/Open_University
68,69,United States,Arizona State University,1885,Public,194000,https://en.wikipedia.org/wiki/Arizona_State_Un...
69,70,Uruguay,University of the Republic,1949,Public,144108,https://en.wikipedia.org/wiki/University_of_th...
